# Visualization of results

Load modules and constants

In [1]:
import shutil
import os
import src.utils as utils
from src.data.dataloader import mtlDataModule

from src.data.manifests import generate_manifest

import torchvision.transforms as T
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
from PIL import Image
from pathlib import Path

from src.utils import load_yaml
import torch

ROOT_DIR = "/home/bnbj/repos/multitask-mayhem"

os.chdir(ROOT_DIR)

CLASS_LOOKUP = load_yaml(ROOT_DIR+"/configs/class_lookup.yaml")

/home/bnbj/miniconda3/envs/multitask-mayhem/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get config to initialize `pl.DataModule` and create dataloaders

In [2]:
config_file = ROOT_DIR + "/configs/debug_foo.yaml"


data_module = mtlDataModule(config_path=config_file)

seg, det = False, False
if data_module.config["model"] in ["fasterrcnn", "fasterrcnn_mobilenetv3", "ssdlite"]:
    det = True
elif data_module.config["model"] in ["deeplabv3"]:
    seg = True

data_module.config["batch_size"] = 1
data_module.config["num_workers"] = 0
data_module.config["shuffle"] = False
print(torch.cuda.is_available())

data_module.prepare_data()
data_module.setup(stage="fit")
train_dataloader = data_module.train_dataloader()
data_module.setup(stage="validate")
valid_dataloader = data_module.val_dataloader()
data_module.setup(stage="test")
test_dataloader = data_module.test_dataloader()

100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


True


Prepare landing folder and label names

In [5]:
sanity_check = ROOT_DIR+"/notebooks/sanity_check"

if os.path.exists(sanity_check):
    shutil.rmtree(sanity_check)
    
os.makedirs(sanity_check, exist_ok=True)

## Sanity check labels

Loop through images and draw labels (use show if not on HPC)

In [6]:
for i, batch in enumerate(train_dataloader):
    image, targets = batch
    
    if det:
        boxes = targets[0]["boxes"]
        labels = targets[0]["labels"]
        masks = targets[0]["masks"]
        label_names = [CLASS_LOOKUP["bbox_rev"][label.item()] for label in labels]
    elif seg:
        masks = targets[0]

    
    img = image[0].mul(255).type(torch.uint8)

    if det:
        drawn_image = draw_bounding_boxes(img, boxes, label_names)
        drawn_image = draw_segmentation_masks(drawn_image, masks, alpha=0.5, colors="green")
    elif seg:
        drawn_image = draw_segmentation_masks(img, masks, alpha=0.5, colors="green")
    
    image_pil = T.ToPILImage()(drawn_image)
    image_pil.save(sanity_check+"/{}.png".format(i))
    # image_pil.show()

Cleanup folder

In [5]:
test_inference = ROOT_DIR+"/notebooks/test_inference"

if os.path.exists(test_inference):
    shutil.rmtree(test_inference)

os.makedirs(test_inference, exist_ok=True)

In [6]:
from src.models.model_loader import ModelLoader

config_file = ROOT_DIR + "/models/ssdlite_baseline_22-12-04T185023/ssdlite_baseline_22-12-04T185023.yaml"
model_folder = str(Path(config_file).parents[0])
print(model_folder)
config = utils.load_yaml(config_file)

seg, det = False, False
if config["model"] in ["fasterrcnn", "fasterrcnn_mobilenetv3", "ssdlite"]:
    det = True
elif config["model"] in ["deeplabv3"]:
    seg = True

model_loader = ModelLoader(config=config)

model = model_loader.grab_model()

model.load_state_dict(torch.load(model_folder+"/weights/best.pth", map_location=torch.device('cpu')))

/home/bnbj/repos/multitask-mayhem/models/ssdlite_baseline_22-12-04T185023


FileNotFoundError: [Errno 2] No such file or directory: '/home/bnbj/repos/multitask-mayhem/models/ssdlite_baseline_22-12-04T185023/ssdlite_baseline_22-12-04T185023.yaml'

In [ ]:
for i, batch in enumerate(test_dataloader):
    image, targets = batch
    
    if det:
        boxes = targets[0]["boxes"]
        labels = targets[0]["labels"]
        scores = targets[0]["scores"]

        score_mask = scores > 0.7

        boxes_filtered = boxes[score_mask]
        labels_filtered = labels[score_mask]
        scores_filtered = scores[score_mask]


        label_names = [CLASS_LOOKUP["bbox_rev"][label.item()] for label in labels_filtered]
    elif seg:
        masks = targets[0]

    
    img = image[0].mul(255).type(torch.uint8)

    if det:
        drawn_image = draw_bounding_boxes(img, boxes_filtered, label_names)
    elif seg:
        drawn_image = draw_segmentation_masks(img, masks, alpha=0.5, colors="green")
    
    image_pil = T.ToPILImage()(drawn_image)
    image_pil.save(test_inference+"/{}.png".format(i))
    # image_pil.show()

KeyError: 'scores'